In [1]:
#raspa tela
import requests
from bs4 import BeautifulSoup
#dataframe
import pandas as pd
#gráficos
import matplotlib.pyplot as plt
#data e hora.
import datetime as dtm
from datetime import date as dt
import time
import re
import numpy as np

# Informando período

In [2]:
#definir quais anos você quer analisar
AnoInicial=int(input("Informe o Ano inicial: "))
AnoFinal=int(input("Informe o Ano final: "))
print("Lembrando que quanto mais anos, mais demorado será a extração")

Informe o Ano inicial:  2018
Informe o Ano final:  2023


Lembrando que quanto mais anos, mais demorado será a extração


# Processo de Web Scraping

In [3]:
tempo_inicio = time.time()
linkRaiz = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a'
listaLinks = []
ListaFinal = []
jogo=[]
Info = []
Rodada = []
j = []
x = []

#Definindo Range e iniciando o BeautifulSoup
for ano in range(AnoInicial, AnoFinal+1):
    for nj in range(1,381):
        linkJogo = f"{linkRaiz}/{ano}/{nj}"
        listaLinks.append(linkJogo)
        valorweb = requests.get(linkJogo)
        soup = BeautifulSoup(valorweb.content, 'html.parser')

#Pegando informações referente ao jogo e adicionando as listas  
        title = soup.title.text
        jogos=soup.find(class_='color-white block text-1').text
        jogo.append(jogos)
        ListaFinal.append(title)
        #pegando informações sobre o Local do jogo
        localdthr_jogo = soup.find_all(class_='text-2 p-r-20')
        Texto = [element.get_text(strip=True) for element in localdthr_jogo]
        Info.append(Texto)
        #Numerojogo
        jogos=soup.find(class_='color-white block text-1').text
        jogos = jogos.split(':')[1].strip()
        
        #Lista de gol por jogador
        GolJog = soup.find_all(class_='hidden-sm hidden-md hidden-lg m-t-20')
        i = [element.get_text(strip=True) for element in GolJog]
        j.extend(i)

        #lista de gol por time
        captTimes = soup.find_all(class_='time-nome color-white')
        Texto = [element.get_text(strip=True) for element in captTimes]
        x.extend(Texto)
           
#Matriz de Gols de jogadores por time
matriz = [[i_item, Texto_item] for i_item, Texto_item in zip(j,x) if j and x != '']

#tempo de execução
tempo_final = time.time()
tempo_decorrido = tempo_final - tempo_inicio


print(f'Tempo decorrido: {tempo_decorrido:.2f} segundos')

Tempo decorrido: 2294.69 segundos


# Iniciar o Processo de DataFrame para acompanhamento dos Jogos

In [4]:
Estadio = []
Cidade = []
Estado = []
Data = []
Horario = []
for item in Info:
    if len(item)>2:
            texto = item[0]  # O nome do estádio está na primeira posição de cada item
            padrao_estadio = r'^(.*?) -'
            resultado_estadio = re.search(padrao_estadio, texto)
            if resultado_estadio:
                nome_estadio = resultado_estadio.group(1)
                Estadio.append(nome_estadio)
            padrao_cidade = r'- (.*?) -'
            resultado_cidade = re.search(padrao_cidade, texto)
            if resultado_cidade:
                nome_cidade = resultado_cidade.group(1)
                Cidade.append(nome_cidade)
            padrao_estado = r'- ([A-Z]{2})$'
            resultado_estado = re.search(padrao_estado, texto)
            if resultado_estado:
                estado = resultado_estado.group(1)
                Estado.append(estado)
            texto1 = item[1]
            padrao_data = r'\w+, \d+ de \w+ de \d+'
            datas_encontradas = re.search(padrao_data, texto1)
            if datas_encontradas:
                data = datas_encontradas.group(0)
                Data.append(data)
    else:
        Estadio.append("Não definido")
        Cidade.append("Não definido")
        Estado.append("Não definido")
        Data.append("Não definido")
        
for item in Info:
    if len(item) > 2:
        horario = item[2]
        if horario != "A definir":
            Horario.append(horario)
        else:
            Horario.append("Não definido")
    else:
        Horario.append("Não definido")
len(Horario)

2280

In [5]:
#Coletar Número Jogo
NumeroJogo=[]
for i in jogo:
    j=i.split('Jogo:')[1].strip()
    NumeroJogo.append(j)


#Coletar Time da Casa
time_casa = []
UF_Time_Mandante = []
for partidas in ListaFinal:
    times = partidas.split(' x ')
    timeCasa = times[0].strip()  # Remover espaços extras no início e no final
    nome_time_casa = timeCasa.split(' - ')[0].strip()  # Separar o nome do time da parte do estado
    Estado= timeCasa.split(' - ')[1].strip()
    Estado = Estado.split(' ')[0]
    time_casa.append(nome_time_casa )
    UF_Time_Mandante.append(Estado)


#coletar Visitante
time_visitante = []
UF_Time_Visitante = []
for partidas in ListaFinal:
    times = partidas.split(' x ')
    timeVisitante = times[1].strip()  # Remover espaços extras no início e no final
    nome_time_visitante = timeVisitante.split(' - ')[0].strip() 
    nome_time_visitante1 = nome_time_visitante[2:]  # Remover o número no início
    time_visitante.append(nome_time_visitante1)
    Estado = timeVisitante.split('-')[1].strip()
    UF_Time_Visitante.append(Estado)



#coletar gols mandante
gols_mandante = []

for gols in ListaFinal:
    gol_mandante = gols.split(' x ')[0].strip()  # Extrair os gols do time da casa
    time = gol_mandante.split(' - ')[1].strip()
    time=time[3:]
    gols_mandante.append(time)

#Coletar gols visitante
gols_visitante=[]
for gols in ListaFinal:
    gols_visitantes=gols.split(' x ')[1].strip()
    gols_visitantes=gols_visitantes.split(' - ')[0].strip()
    gols_visitantes=gols_visitantes.split(' ')[0]
    gols_visitante.append(gols_visitantes)

#Coletar Ano
anoJogo=[]
for i in ListaFinal:
    Ano_Jogos=i.split(' x ')[1].strip()
    Ano_Jogos=Ano_Jogos.split(' - ')[3]
    Ano_Jogos=Ano_Jogos[-4:]
    anoJogo.append(Ano_Jogos)

In [6]:
#Determinar quem venceu
def vitoria(gols_mandante, gols_visitante):
    if gols_mandante > gols_visitante:
        return 'Mandante'
    elif gols_mandante < gols_visitante:
        return 'Visitante'
    else:
        return 'Empate'
resultado=[vitoria(gols_mandante,gols_visitante) for gols_mandante,gols_visitante in zip(gols_mandante,gols_visitante)]

In [7]:
#Criando o DataFrame 
data = {'NumeroJogo':NumeroJogo,'Data':Data,'Hora':Horario,'Estadio':Estadio,'Cidade':Cidade,'UF':Estado, 'Quem Venceu?':resultado, 'AnoJogo':anoJogo, 'Time Mandante':time_casa,'UF Mandante': UF_Time_Mandante , 'Gols Mandante':gols_mandante,'Time Visitante':time_visitante,'UF Visitante': UF_Time_Visitante,'Gols Visitante':gols_visitante}
df = pd.DataFrame(data)
df

,NumeroJogo,Data,Hora,Estadio,Cidade,UF,Quem Venceu?,AnoJogo,Time Mandante,UF Mandante,Gols Mandante,Time Visitante,UF Visitante,Gols Visitante
0,1,"Sábado, 14 de Abril de 2018",16:00,Mineirão,Belo Horizonte,PR,Visitante,2018,Cruzeiro,MG,0,Grêmio,RS,1
1,2,"Domingo, 15 de Abril de 2018",19:00,Arena da Baixada,Curitiba,PR,Mandante,2018,Athletico Paranaense,PR,5,Chapecoense,SC,1
2,3,"Domingo, 15 de Abril de 2018",11:00,Independência,Belo Horizonte,PR,Mandante,2018,America Fc,MG,3,Sport,PE,0
3,4,"Sábado, 14 de Abril de 2018",19:00,Manoel Barradas,Salvador,PR,Empate,2018,Vitória,BA,2,Flamengo,RJ,2
4,5,"Domingo, 15 de Abril de 2018",16:00,São Januário,Rio de Janeiro,PR,Mandante,2018,Vasco da Gama,RJ,2,Atlético,MG,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,376,Não definido,Não definido,Não definido,Não definido,PR,Visitante,2023,Cruzeiro Saf,MG,,lmeiras,SP,Palmeiras
2276,377,Não definido,Não definido,Não definido,Não definido,PR,Visitante,2023,Internacional,RS,,tafogo,RJ,Botafogo
2277,378,Não definido,Não definido,Não definido,Não definido,PR,Visitante,2023,Coritiba S.a.f.,PR,,rinthians,SP,Corinthians
2278,379,Não definido,Não definido,Não definido,Não definido,PR,Visitante,2023,Bahia,BA,,lético Mineiro,MG,Atlético


In [8]:
#Excluindo jogos que ainda não ocorreram 
df['Gols Visitante'] = pd.to_numeric(df['Gols Visitante'], errors='coerce').astype('Int64')
df['Gols Mandante'] = pd.to_numeric(df['Gols Mandante'], errors='coerce').astype('Int64')
df['NumeroJogo'] = pd.to_numeric(df['NumeroJogo'], errors='coerce').astype('Int64')
df['Rodada'] = df['NumeroJogo'].apply(lambda num_jogo: int(num_jogo / 10) if num_jogo % 10 == 0 else int(num_jogo // 10) + 1)
df['Time Mandante'] = df['Time Mandante'].str.replace('.', '')
df['Time Visitante'] = df['Time Visitante'].str.replace('.', '')
df = df.dropna(subset=['Gols Visitante'])
X = ['NumeroJogo', 'Data','Hora','Estadio', 'Cidade','UF','Quem Venceu?', 'AnoJogo', 'Rodada', 'Time Mandante', 'UF Mandante', 'Gols Mandante', 'Time Visitante', 'UF Visitante', 'Gols Visitante']
df = df[X]
df

,NumeroJogo,Data,Hora,Estadio,Cidade,UF,Quem Venceu?,AnoJogo,Rodada,Time Mandante,UF Mandante,Gols Mandante,Time Visitante,UF Visitante,Gols Visitante
0,1,"Sábado, 14 de Abril de 2018",16:00,Mineirão,Belo Horizonte,PR,Visitante,2018,1,Cruzeiro,MG,0,Grêmio,RS,1
1,2,"Domingo, 15 de Abril de 2018",19:00,Arena da Baixada,Curitiba,PR,Mandante,2018,1,Athletico Paranaense,PR,5,Chapecoense,SC,1
2,3,"Domingo, 15 de Abril de 2018",11:00,Independência,Belo Horizonte,PR,Mandante,2018,1,America Fc,MG,3,Sport,PE,0
3,4,"Sábado, 14 de Abril de 2018",19:00,Manoel Barradas,Salvador,PR,Empate,2018,1,Vitória,BA,2,Flamengo,RJ,2
4,5,"Domingo, 15 de Abril de 2018",16:00,São Januário,Rio de Janeiro,PR,Mandante,2018,1,Vasco da Gama,RJ,2,Atlético,MG,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,336,"Domingo, 12 de Novembro de 2023",18:30,ARENA MRV,Belo Horizonte,PR,Mandante,2023,34,Atlético Mineiro,MG,2,Goiás,GO,1
2236,337,"Domingo, 12 de Novembro de 2023",16:00,Arena do Grêmio,Porto Alegre,PR,Visitante,2023,34,Grêmio,RS,0,Corinthians,SP,1
2237,338,"Sábado, 11 de Novembro de 2023",16:00,Durival Britto,Curitiba,PR,Mandante,2023,34,Coritiba Saf,PR,1,Cruzeiro Saf,MG,0
2238,339,"Domingo, 12 de Novembro de 2023",18:30,Arena Fonte Nova,Salvador,PR,Empate,2023,34,Bahia,BA,1,Athletico Paranaense,PR,1


### Acompanhar desempenho dos times Individualmente

In [9]:
#Copiar Dataframe anterior
dx_local = df[['NumeroJogo', 'Time Mandante','UF Mandante', 'Gols Mandante', 'Quem Venceu?','Estadio', 'Cidade','UF', 'Hora', 'Data', 'AnoJogo']].copy()
dx_visitante = df[['NumeroJogo', 'Time Visitante','UF Visitante', 'Gols Visitante', 'Quem Venceu?', 'Estadio', 'Cidade','UF', 'Hora', 'Data','AnoJogo']].copy()
dx_local.columns = ['NumeroJogo', 'Time','UF Time', 'Gols', 'Quem Venceu?','Estadio', 'Cidade','UF', 'Hora', 'Data', 'AnoJogo']
dx_local['Mandante/Visitante'] = 'Mandante'
dx_visitante.columns = ['NumeroJogo', 'Time','UF Time', 'Gols', 'Quem Venceu?', 'Estadio', 'Cidade','UF', 'Hora', 'Data', 'AnoJogo']
dx_visitante['Mandante/Visitante'] = 'Visitante'
#concatenar
dx = pd.concat([dx_local, dx_visitante], ignore_index=True)
#Demonstrar pontuação
dx['Pontos'] = dx.apply(lambda row: 3 if row['Quem Venceu?'] == row['Mandante/Visitante'] else (1 if row['Quem Venceu?'] == 'Empate' else 0), axis=1)
dx = dx.drop('Quem Venceu?', axis=1)
dx

,NumeroJogo,Time,UF Time,Gols,Estadio,Cidade,UF,Hora,Data,AnoJogo,Mandante/Visitante,Pontos
0,1,Cruzeiro,MG,0,Mineirão,Belo Horizonte,PR,16:00,"Sábado, 14 de Abril de 2018",2018,Mandante,0
1,2,Athletico Paranaense,PR,5,Arena da Baixada,Curitiba,PR,19:00,"Domingo, 15 de Abril de 2018",2018,Mandante,3
2,3,America Fc,MG,3,Independência,Belo Horizonte,PR,11:00,"Domingo, 15 de Abril de 2018",2018,Mandante,3
3,4,Vitória,BA,2,Manoel Barradas,Salvador,PR,19:00,"Sábado, 14 de Abril de 2018",2018,Mandante,1
4,5,Vasco da Gama,RJ,2,São Januário,Rio de Janeiro,PR,16:00,"Domingo, 15 de Abril de 2018",2018,Mandante,3
...,...,...,...,...,...,...,...,...,...,...,...,...
4465,336,Goiás,GO,1,ARENA MRV,Belo Horizonte,PR,18:30,"Domingo, 12 de Novembro de 2023",2023,Visitante,0
4466,337,Corinthians,SP,1,Arena do Grêmio,Porto Alegre,PR,16:00,"Domingo, 12 de Novembro de 2023",2023,Visitante,3
4467,338,Cruzeiro Saf,MG,0,Durival Britto,Curitiba,PR,16:00,"Sábado, 11 de Novembro de 2023",2023,Visitante,0
4468,339,Athletico Paranaense,PR,1,Arena Fonte Nova,Salvador,PR,18:30,"Domingo, 12 de Novembro de 2023",2023,Visitante,1


## Informação de Gols por jogador e time

### Alguns jogos os times não liberaram a escalação, então falta informação no site da CBF e consequentemente faltaram alguns gols dos atletas
#### ex: jogo 5 (2023) - RB Bragantino VS Bahia 

In [10]:
#Criando matriz para iterar jogos, times, jogador e jogos
x = np.array(matriz)
y = np.array(NumeroJogo)
j = np.array(anoJogo)
matriz1_reshaped = x.reshape(-1, 2, 2)

resultado = []

# Use um índice para rastrear o número do jogo da matriz 2
indice_matriz2 = 0

# Itere sobre a matriz 1 agrupando os registros em jogos e adicionando o número do jogo

#pegando pares de elementos da lista x, combinando-os com informações de j e y e em seguida, adicionando essas informações a uma lista resultado
for i in range(0, len(x), 2):
    jogo = x[i:i + 2]
    ano = j[indice_matriz2]  
    numero_jogo = y[indice_matriz2]
    resultado.append([numero_jogo, jogo, ano]) 
    indice_matriz2 += 1


numeros_jogo = []
detalhes_jogo = []
anos = []

# Itere sobre o resultado
for item in resultado:
    numeros_jogo.append(item[0])
    detalhes_jogo.append(item[1])
    anos.append(item[2])


In [11]:
lista_de_listas = []
# Itere pelos dados originais e crie uma nova lista para cada elemento
for elemento in detalhes_jogo:
    nova_lista = [item for sublist in elemento for item in sublist]
    lista_de_listas.append(nova_lista)
lista_de_listas = list(zip(numeros_jogo, lista_de_listas, anos))


In [12]:
detalhe = []
jogo = []
ano=[]

for elemento in lista_de_listas:
    numero_jogo = elemento[0]
    gols_detalhes = elemento[1]
    anos=elemento[2]
    detalhe.extend(gols_detalhes)
   
    contagem_t = 0
    for texto in gols_detalhes:
        contagem_t += texto.count('T)')
    
    jogo.extend([numero_jogo] * contagem_t)
    ano.extend([anos] * contagem_t)

nomes_jogadores = []
nomes_times = []      
for i, dado in enumerate(detalhe):
    
    if i % 2 == 0:
        nomes = dado.split(')')
        nomes = [nome.strip() + ')' for nome in nomes if nome.strip()]
        nomes_jogadores.extend(nomes)
    else:
        # Multiplicar o nome do time pelo número de jogadores
        nomes_times.extend([dado] * len(nomes))

In [13]:
#Criando Regex através de Ferramenta de identificação de padrões 
padrao = r'([\w\s]+)\s(\d{1,2}\+?\d*)\'\s\((\d{1,2}[+-]?\d*\w*)\)'
# Lista para armazenar os resultados
resultados = []
Atleta = []
Minutos = []
Tempo = []
Minutos_Acrescido = []
# Loop sobre as strings
for s in nomes_jogadores:
    correspondencias = re.search(padrao, s)
    if correspondencias:
        nome = correspondencias.group(1)
        minuto = correspondencias.group(2)
        tempo = correspondencias.group(3)
        resultados.append({'Nome': nome, 'Minuto': minuto, 'Tempo': tempo})
        Atleta.append(nome)
        Minutos.append(minuto)
        Tempo.append(tempo)
nomeOrientado=[]
for i in Atleta:
    x = i.capitalize()
    nomeOrientado.append(x)
    
    

for i in Minutos:
    x = i.split('+')
    z = int(x[0])
    
    for acrescimo in x[1:]:
        z += int(acrescimo)

    Minutos_Acrescido.append(z)

nomeOrientado


dy = pd.DataFrame({'Nome': nomeOrientado, 'Minuto':Minutos_Acrescido,'Tempo':Tempo, 'Time': nomes_times, 'Jogo':jogo, 'Ano': ano})
#Exiba o DataFrame
dy

,Nome,Minuto,Tempo,Time,Jogo,Ano
0,Andre felipe,9,2ºT,Grêmio - RS,1,2018
1,Pablo,6,2ºT,Athletico Paranaense - PR,2,2018
2,Nikao,13,2ºT,Athletico Paranaense - PR,2,2018
3,Carleto,23,2ºT,Athletico Paranaense - PR,2,2018
4,Rossetto,42,2ºT,Athletico Paranaense - PR,2,2018
...,...,...,...,...,...,...
5205,Everaldo,28,2ºT,Bahia - BA,339,2023
5206,Canobbio,42,2ºT,Athletico Paranaense - PR,339,2023
5207,Deyverson,5,1ºT,Cuiabá Saf - MT,340,2023
5208,Pitta,46,2ºT,Cuiabá Saf - MT,340,2023


In [16]:
dy['Jogo'] = pd.to_numeric(dy['Jogo'], errors='coerce').astype('Int64')
dy['Rodada'] = dy['Jogo'].apply(lambda num_jogo: int(num_jogo / 10) if num_jogo % 10 == 0 else int(num_jogo // 10) + 1)
dy

In [18]:
#Exportando Para Excel
with pd.ExcelWriter(f"CampeonatoBrasileiro_{AnoInicial}-{AnoFinal}_SerieA.xlsx") as writer:
    # Escreva cada DataFrame em uma planilha diferente
    df.to_excel(writer, sheet_name='Jogos', index=False)
    dx.to_excel(writer, sheet_name='DesempenhoIndividual', index=False)
    dy.to_excel(writer, sheet_name='Gols', index=False)
